# Temporal Analysis: 50 Years of Livestock Farming in Nepal (1972-2021)

## Synthetic Historical Data Generation and Trend Analysis

**Author:** ML Assignment Project  
**Date:** December 2025

---

### Overview

This notebook extends the livestock farming vulnerability analysis by:
1. **Generating synthetic historical data** for 50 years (1972-2021) based on 2021 baseline
2. **Analyzing temporal trends** in agricultural practices across Nepal's districts
3. **Identifying structural changes** in farming systems over time
4. **Clustering evolution** - how farming profiles have changed over decades

## 1. Setup and Library Installation

### 1.1 Install Required Libraries (for Google Colab)


In [ ]:
# Install required libraries (for Google Colab)
!pip install geopandas folium mapclassify -q


### 1.2 Import Libraries


In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning libraries
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    silhouette_score
)
from sklearn.linear_model import LinearRegression

# Geographic visualization (optional - will handle gracefully if not available)
try:
    import geopandas as gpd
    GEOPANDAS_AVAILABLE = True
except ImportError:
    GEOPANDAS_AVAILABLE = False
    print("Note: geopandas not available. Map visualizations will be skipped.")
    print("Install with: pip install geopandas")

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ All libraries imported successfully!")


## 2. Load 2021 Baseline Data from Files\n

In [ ]:
# Detect environment and set up data loading
import os

# Check if running in Google Colab
IN_COLAB = 'google.colab' in str(get_ipython()) if 'get_ipython' in dir() else False

if IN_COLAB:
    print("Running in Google Colab environment")
else:
    print("Running in local environment")

In [ ]:
# Define file paths
# Update these paths according to your setup

if IN_COLAB:
    # Select/upload CSV files in Colab (file picker)
    from google.colab import files
    print("Please select the three CSV files (you can multi-select):")
    print("  - table-1.-number-of-holdings-and-area-by-district-.csv")
    print("  - table-4.1.-number-area-number-of-holdings-reporting-and-area-irrigated-by-source-of-irrigation-a.csv")
    print("  - table-17.-number-of-holdings-livestocks-and-poultry-by-districts.csv")
    uploaded = files.upload()

    def pick_file(keyword, fallback=None):
        for fname in uploaded.keys():
            if keyword.lower() in fname.lower():
                return fname
        if fallback and fallback in uploaded:
            return fallback
        return list(uploaded.keys())[0]

    holdings_area_path = pick_file('holdings-and-area', 'table-1.-number-of-holdings-and-area-by-district-.csv')
    irrigation_path = pick_file('irrigation', 'table-4.1.-number-area-number-of-holdings-reporting-and-area-irrigated-by-source-of-irrigation-a.csv')
    livestock_path = pick_file('livestocks-and-poultry', 'table-17.-number-of-holdings-livestocks-and-poultry-by-districts.csv')

    print(f"Using files -> holdings: {holdings_area_path}, irrigation: {irrigation_path}, livestock: {livestock_path}")
else:
    # Local paths - update as needed
    base_path = '/Users/eklakdangaura/College/ML/Assignment/Datasets/'
    holdings_area_path = base_path + 'table-1.-number-of-holdings-and-area-by-district-.csv'
    irrigation_path = base_path + 'table-4.1.-number-area-number-of-holdings-reporting-and-area-irrigated-by-source-of-irrigation-a.csv'
    livestock_path = base_path + 'table-17.-number-of-holdings-livestocks-and-poultry-by-districts.csv'

print("File paths configured")

In [ ]:
# Load the three datasets from CSV files
print("Loading datasets from files...")

# Dataset 1: Holdings and Area by District
df_land = pd.read_csv(holdings_area_path, sep='\t')
print(f"1. Holdings & Area Dataset: {df_land.shape[0]} rows, {df_land.shape[1]} columns")

# Dataset 2: Irrigation Data
df_irr = pd.read_csv(irrigation_path, sep='\t')
print(f"2. Irrigation Dataset: {df_irr.shape[0]} rows, {df_irr.shape[1]} columns")

# Dataset 3: Livestock Data
df_live = pd.read_csv(livestock_path)
print(f"3. Livestock Dataset: {df_live.shape[0]} rows, {df_live.shape[1]} columns")

print("\n✓ All datasets loaded successfully from files!")

### 2.1 Exploratory Data Analysis (EDA)


In [ ]:
# Explore Holdings & Area Dataset
print("=" * 60)
print("DATASET 1: Holdings and Area by District")
print("=" * 60)
print(f"\nShape: {df_land.shape}")
print(f"\nColumns: {df_land.columns.tolist()}")
print("\nData Types:")
print(df_land.dtypes)
print("\nFirst 5 rows:")
display(df_land.head())


In [ ]:
# Explore Irrigation Dataset
print("=" * 60)
print("DATASET 2: Irrigation Data")
print("=" * 60)
print(f"\nShape: {df_irr.shape}")
print(f"\nColumns: {df_irr.columns.tolist()}")
print("\nData Types:")
print(df_irr.dtypes)
print("\nFirst 5 rows:")
display(df_irr.head())


In [ ]:
# Explore Livestock Dataset
print("=" * 60)
print("DATASET 3: Livestock Data")
print("=" * 60)
print(f"\nShape: {df_live.shape}")
print(f"\nColumns: {df_live.columns.tolist()}")
print("\nData Types:")
print(df_live.dtypes)
print("\nFirst 5 rows:")
display(df_live.head())


In [ ]:
# Summary Statistics for all datasets
print("=" * 60)
print("SUMMARY STATISTICS - ORIGINAL DATASETS")
print("=" * 60)

print("\n1. Holdings & Area Dataset Statistics:")
display(df_land.describe())

print("\n2. Irrigation Dataset Statistics:")
display(df_irr.describe())

print("\n3. Livestock Dataset Statistics:")
display(df_live.describe())

# Visualize key distributions from original data
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Holdings distribution
axes[0].hist(df_land['Number of holdings'].dropna(), bins=20, color='steelblue', edgecolor='black')
axes[0].set_title('Distribution of Holdings by District', fontweight='bold')
axes[0].set_xlabel('Number of Holdings')
axes[0].set_ylabel('Frequency')

# Total area distribution
axes[1].hist(df_land['Total  area (ha)'].dropna(), bins=20, color='forestgreen', edgecolor='black')
axes[1].set_title('Distribution of Total Area (ha)', fontweight='bold')
axes[1].set_xlabel('Area (hectares)')
axes[1].set_ylabel('Frequency')

# Livestock holdings distribution
axes[2].hist(df_live['Number of holdings reporting livestock'].dropna(), bins=20, color='coral', edgecolor='black')
axes[2].set_title('Distribution of Livestock Holdings', fontweight='bold')
axes[2].set_xlabel('Number of Holdings')
axes[2].set_ylabel('Frequency')

plt.suptitle('Original Dataset Distributions (2021 Baseline)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('original_data_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Original dataset visualizations saved")

### 2.2 Data Preprocessing and District Name Standardization


In [ ]:
# Standardize district names and merge datasets
def standardize_district_name(name):
    if pd.isna(name):
        return name
    return str(name).strip().lower()

# Clean district names
df_land['district_clean'] = df_land['Districts'].apply(standardize_district_name)
df_irr['district_clean'] = df_irr['Districts'].apply(standardize_district_name)
df_live['district_clean'] = df_live['District'].apply(standardize_district_name)

# Select relevant columns
df_land_select = df_land[['district_clean', 'Districts', 'Number of holdings', 
                          'Total wet  area (ha)', 'Total dry  area (ha)', 'Total  area (ha)']].copy()
df_land_select.columns = ['district_clean', 'Districts', 'Number of holdings', 
                          'wet_area_ha', 'dry_area_ha', 'total_area_ha']

df_irr_select = df_irr[['district_clean', 'No. of holdings reporting irrigation', 
                        'Total area (ha) of irrigation']].copy()
df_irr_select.columns = ['district_clean', 'holdings_with_irrigation', 'irrigated_area_ha']

# Handle livestock columns
livestock_cols = ['district_clean', 'Total number of holdings', 'Number of holdings reporting livestock',
                  'No. of  cattles', 'No. of buffalo', 'No. of goat/chyangra', 
                  'No. of pigs/boar', 'No. of poultry(chicken)', 'No. of sheep']
df_live_select = df_live[livestock_cols].copy()
df_live_select.columns = ['district_clean', 'total_holdings_livestock', 'holdings_with_livestock',
                          'num_cattle', 'num_buffalo', 'num_goats', 'num_pigs', 'num_poultry', 'num_sheep']

# Fill missing values
for col in df_live_select.columns:
    if col != 'district_clean':
        df_live_select[col] = pd.to_numeric(df_live_select[col], errors='coerce').fillna(0)

# Merge all datasets
df_merged = df_land_select.merge(df_irr_select, on='district_clean', how='outer').merge(
    df_live_select, on='district_clean', how='outer')

# Fill any remaining missing values
numeric_cols = df_merged.select_dtypes(include=[np.number]).columns
df_merged[numeric_cols] = df_merged[numeric_cols].fillna(0)

print(f"Merged dataset: {df_merged.shape[0]} districts, {df_merged.shape[1]} columns")
print(f"\nColumns: {df_merged.columns.tolist()}")

In [ ]:
# Check for district name mismatches across datasets
holdings_districts = set(df_land['district_clean'].dropna())
irrigation_districts = set(df_irr['district_clean'].dropna())
livestock_districts = set(df_live['district_clean'].dropna())

# Find common districts
common_districts = holdings_districts & irrigation_districts & livestock_districts
print(f"District Name Verification:")
print("=" * 60)
print(f"Holdings dataset: {len(holdings_districts)} districts")
print(f"Irrigation dataset: {len(irrigation_districts)} districts")
print(f"Livestock dataset: {len(livestock_districts)} districts")
print(f"\nCommon districts across all datasets: {len(common_districts)}")

# Check for mismatches
all_districts = holdings_districts | irrigation_districts | livestock_districts
if len(all_districts) != len(common_districts):
    print("\n⚠️ Districts with potential mismatches:")
    only_holdings = holdings_districts - common_districts
    only_irrigation = irrigation_districts - common_districts
    only_livestock = livestock_districts - common_districts
    
    if only_holdings:
        print(f"  Only in Holdings: {only_holdings}")
    if only_irrigation:
        print(f"  Only in Irrigation: {only_irrigation}")
    if only_livestock:
        print(f"  Only in Livestock: {only_livestock}")
else:
    print("\n✓ All districts match across datasets!")


### 2.3 Merge Datasets and Handle Missing Values


In [ ]:
# Feature Engineering for 2021 baseline
df_merged['Avg_Land_Size'] = df_merged['total_area_ha'] / df_merged['Number of holdings']
df_merged['Pct_Irrigated'] = (df_merged['irrigated_area_ha'] / df_merged['total_area_ha']) * 100
df_merged['Cattle_per_HH'] = df_merged['num_cattle'] / df_merged['Number of holdings']
df_merged['Buffalo_per_HH'] = df_merged['num_buffalo'] / df_merged['Number of holdings']
df_merged['Goat_per_HH'] = df_merged['num_goats'] / df_merged['Number of holdings']
df_merged['Pig_per_HH'] = df_merged['num_pigs'] / df_merged['Number of holdings']
df_merged['Poultry_per_HH'] = df_merged['num_poultry'] / df_merged['Number of holdings']

# Handle infinite values
df_merged = df_merged.replace([np.inf, -np.inf], 0)

# Create 2021 baseline
base_cols = ['Districts', 'district_clean', 'Avg_Land_Size', 'Pct_Irrigated', 
             'Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Pig_per_HH', 'Poultry_per_HH']

# Filter out rows without district name
df_2021 = df_merged[base_cols].copy()
df_2021 = df_2021.dropna(subset=['Districts'])
df_2021['Year'] = 2021

print(f"2021 baseline dataset prepared: {df_2021.shape[0]} districts")
print(f"\n2021 Baseline Statistics:")
print(df_2021.describe().round(2))

In [ ]:
# Check for missing values after merge
print("Missing Values Analysis:")
print("=" * 60)
missing = df_merged.isnull().sum()
if missing.any():
    print("Columns with missing values:")
    print(missing[missing > 0])
else:
    print("✓ No missing values in merged dataset!")

print(f"\nMerged dataset shape: {df_merged.shape}")
print(f"Number of districts: {df_merged['district_clean'].nunique()}")
print("\nMerged dataset preview:")
display(df_merged.head())


### 2.4 Feature Engineering and Visualization


In [ ]:
# Feature Engineering Visualizations - Using 2021 Baseline (before generating temporal data)
print("Visualizing Engineered Features for 2021 Baseline...")

# Define engineered features
eng_features = ['Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Buffalo_per_HH', 
                'Goat_per_HH', 'Pig_per_HH', 'Poultry_per_HH']

# 1. Correlation Heatmap for 2021 Baseline
plt.figure(figsize=(12, 8))
correlation_matrix = df_2021[eng_features].corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, linewidths=0.5, square=True)
plt.title('Correlation Matrix of Engineered Features (2021 Baseline)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('temporal_feature_correlation_2021.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ 2021 Baseline correlation heatmap saved")

In [ ]:
# 2. Distribution of Key Features (2021 Baseline)
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for idx, feature in enumerate(eng_features):
    sns.histplot(df_2021[feature], kde=True, ax=axes[idx], color='steelblue')
    axes[idx].set_title(feature.replace('_', ' ').title(), fontweight='bold')
    axes[idx].set_xlabel('')

# Turn off extra subplots
for idx in range(len(eng_features), len(axes)):
    axes[idx].axis('off')

plt.suptitle('Distribution of Engineered Features (2021 Baseline)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('temporal_feature_distributions_2021.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ 2021 Feature distributions saved")

In [ ]:
# 3. Pairplot for Key Livestock Features (2021 Baseline)
key_features = ['Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Poultry_per_HH']
fig = sns.pairplot(df_2021[key_features], diag_kind='kde', plot_kws={'alpha': 0.6})
fig.fig.suptitle('Pairwise Relationships: Livestock Features (2021 Baseline)', y=1.02, fontsize=14, fontweight='bold')
plt.savefig('temporal_pairplot_2021.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ 2021 Pairplot saved")

print("\n✓ 2021 Baseline feature engineering visualizations complete!")

## 3. Generate Synthetic Historical Data (1972-2020)

In [ ]:
np.random.seed(42)
all_years_data = [df_2021]
current_df = df_2021.copy()

print('Generating 50 years of synthetic data...')
print('Trends: Poultry -8%/yr, Land +0.8%/yr, Irrigation -1%/yr, Pigs -2%/yr')

for year in range(2020, 1971, -1):
    new_df = current_df.copy()
    new_df['Year'] = year
    noise = np.random.uniform(0.98, 1.02, size=len(new_df))
    
    new_df['Poultry_per_HH'] = new_df['Poultry_per_HH'] * 0.92 * noise
    new_df['Avg_Land_Size'] = new_df['Avg_Land_Size'] * 1.008 * noise
    new_df['Pct_Irrigated'] = new_df['Pct_Irrigated'] * 0.99 * noise
    new_df['Pig_per_HH'] = new_df['Pig_per_HH'] * 0.98 * noise
    new_df['Goat_per_HH'] = new_df['Goat_per_HH'] * 1.001 * noise
    new_df['Cattle_per_HH'] = new_df['Cattle_per_HH'] * 1.001 * noise
    new_df['Buffalo_per_HH'] = new_df['Buffalo_per_HH'] * 1.001 * noise
    
    cols = new_df.select_dtypes(include=[np.number]).columns
    new_df[cols] = new_df[cols].clip(lower=0)
    
    all_years_data.append(new_df)
    current_df = new_df

df_long = pd.concat(all_years_data, ignore_index=True)
df_long = df_long.sort_values(['Districts', 'Year']).reset_index(drop=True)

print(f'Generated {len(df_long)} rows (50 years x {len(df_2021)} districts)')
df_long.to_csv('nepal_agriculture_50_years.csv', index=False)
print('Exported to nepal_agriculture_50_years.csv')

### 3.1 EDA on Full 50-Year Dataset

Now that we have the complete 50-year synthetic dataset, let's perform comprehensive exploratory data analysis.

In [ ]:
# Comprehensive EDA on Full 50-Year Dataset
print("=" * 70)
print("EXPLORATORY DATA ANALYSIS: 50-YEAR DATASET (1972-2021)")
print("=" * 70)

# Define features for analysis
features = ['Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Pig_per_HH', 'Poultry_per_HH']

print(f"\nDataset Shape: {df_long.shape}")
print(f"Total Records: {len(df_long):,}")
print(f"Number of Districts: {df_long['Districts'].nunique()}")
print(f"Number of Years: {df_long['Year'].nunique()}")
print(f"Year Range: {df_long['Year'].min()} - {df_long['Year'].max()}")

print("\n" + "=" * 70)
print("FEATURE STATISTICS ACROSS ALL 50 YEARS")
print("=" * 70)
display(df_long[features].describe().round(3))

# Missing values check
print("\n" + "=" * 70)
print("MISSING VALUES ANALYSIS")
print("=" * 70)
missing = df_long.isnull().sum()
if missing.any():
    print("Columns with missing values:")
    print(missing[missing > 0])
else:
    print("✓ No missing values in the 50-year dataset!")

In [ ]:
# Correlation Analysis Across All 50 Years
print("Correlation Analysis on Full 50-Year Dataset")
print("=" * 60)

plt.figure(figsize=(12, 8))
correlation_matrix_full = df_long[features].corr()
mask = np.triu(np.ones_like(correlation_matrix_full, dtype=bool))
sns.heatmap(correlation_matrix_full, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, linewidths=0.5, square=True)
plt.title('Correlation Matrix - Full 50-Year Dataset (1972-2021)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('correlation_matrix_50years.png', dpi=150, bbox_inches='tight')
plt.show()

# Compare correlations between 2021 and full dataset
print("\nCorrelation Comparison: 2021 vs Full 50-Year Dataset")
print("-" * 60)
for i, feat1 in enumerate(features):
    for feat2 in features[i+1:]:
        corr_2021 = df_2021[feat1].corr(df_2021[feat2])
        corr_full = df_long[feat1].corr(df_long[feat2])
        if abs(corr_2021 - corr_full) > 0.1:  # Significant difference
            print(f"{feat1} vs {feat2}: 2021={corr_2021:.3f}, Full={corr_full:.3f}")

In [ ]:
# Distribution Analysis Across All 50 Years
fig, axes = plt.subplots(3, 3, figsize=(16, 14))
axes = axes.flatten()

for idx, feature in enumerate(features):
    ax = axes[idx]
    # Plot distribution for different decades
    decades = [(1972, 1981, 'red', '1970s'), (1992, 2001, 'blue', '1990s'), (2012, 2021, 'green', '2010s')]
    for start, end, color, label in decades:
        subset = df_long[(df_long['Year'] >= start) & (df_long['Year'] <= end)]
        sns.kdeplot(data=subset, x=feature, ax=ax, color=color, label=label, fill=True, alpha=0.2)
    ax.set_title(feature.replace('_', ' ').title(), fontweight='bold')
    ax.legend(fontsize=8)

for idx in range(len(features), len(axes)):
    axes[idx].axis('off')

plt.suptitle('Feature Distribution Evolution: 1970s vs 1990s vs 2010s', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('distribution_evolution_50years.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Distribution evolution plot saved")

In [ ]:
# Box Plot Analysis Across Decades
fig, axes = plt.subplots(2, 4, figsize=(18, 10))
axes = axes.flatten()

# Add decade column for visualization
df_long['Decade'] = (df_long['Year'] // 10) * 10
df_long['Decade_Label'] = df_long['Decade'].astype(str) + 's'

for idx, feature in enumerate(features):
    if idx < len(axes):
        sns.boxplot(x='Decade_Label', y=feature, data=df_long, ax=axes[idx], palette='viridis')
        axes[idx].set_title(feature.replace('_', ' ').title(), fontweight='bold', fontsize=10)
        axes[idx].set_xlabel('')
        axes[idx].tick_params(axis='x', rotation=45)

# Turn off extra subplot
axes[-1].axis('off')

plt.suptitle('Feature Distribution by Decade (1970s-2020s)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('boxplot_by_decade_50years.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Decade-wise boxplot saved")

In [ ]:
# Statistical Summary by Decade
print("=" * 80)
print("STATISTICAL SUMMARY BY DECADE")
print("=" * 80)

decade_summary = df_long.groupby('Decade')[features].agg(['mean', 'std', 'min', 'max'])
print("\nMean Values by Decade:")
display(df_long.groupby('Decade')[features].mean().round(3))

print("\nStandard Deviation by Decade:")
display(df_long.groupby('Decade')[features].std().round(3))

# Calculate decade-over-decade growth rates
print("\n" + "=" * 80)
print("DECADE-OVER-DECADE GROWTH RATES (%)")
print("=" * 80)
decade_means = df_long.groupby('Decade')[features].mean()
growth_rates = decade_means.pct_change() * 100
display(growth_rates.round(2))

### 3.2 Feature Engineering Visualizations on Full 50-Year Dataset

In [ ]:
# Pairplot for Full 50-Year Dataset (sampled for performance)
print("Creating pairplot for 50-year dataset (sampled)...")

# Define era assignment function if not already defined
def assign_era(year):
    if year <= 1985:
        return 'Era 1: Traditional (1972-1985)'
    elif year <= 2000:
        return 'Era 2: Early Modern (1986-2000)'
    else:
        return 'Era 3: Commercial (2001-2021)'

# Sample data for visualization (to avoid slowness)
df_sample = df_long.sample(n=min(2000, len(df_long)), random_state=42)
df_sample['Era'] = df_sample['Year'].apply(assign_era)

key_features_pair = ['Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Poultry_per_HH']
fig = sns.pairplot(df_sample, vars=key_features_pair, hue='Era', diag_kind='kde', 
                   plot_kws={'alpha': 0.5, 's': 20}, palette='Set1')
fig.fig.suptitle('Pairwise Relationships by Era (50-Year Dataset)', y=1.02, fontsize=14, fontweight='bold')
plt.savefig('pairplot_by_era_50years.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Era-wise pairplot saved")

In [ ]:
# Time Series Visualization for Selected Districts
print("Time Series Analysis for Selected Districts...")

# Select top 5 districts by 2021 total livestock
top_districts = df_2021.nlargest(5, 'Poultry_per_HH')['Districts'].tolist()

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, feature in enumerate(features[:6]):
    ax = axes[idx]
    for district in top_districts:
        district_data = df_long[df_long['Districts'] == district].sort_values('Year')
        ax.plot(district_data['Year'], district_data[feature], label=district.strip()[:15], linewidth=1.5)
    ax.set_xlabel('Year')
    ax.set_ylabel(feature.replace('_', ' '))
    ax.set_title(f'{feature.replace("_", " ").title()} Over Time', fontweight='bold')
    ax.legend(fontsize=7, loc='upper left')
    ax.grid(True, alpha=0.3)

plt.suptitle('50-Year Trends for Top 5 Districts (by Poultry)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('district_trends_50years.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ District time series saved")

In [ ]:
# Feature Variance Analysis Over Time
print("Feature Variance Analysis Over Time...")

variance_by_year = df_long.groupby('Year')[features].var()

fig, axes = plt.subplots(2, 4, figsize=(18, 10))
axes = axes.flatten()

for idx, feature in enumerate(features):
    ax = axes[idx]
    ax.plot(variance_by_year.index, variance_by_year[feature], linewidth=2, color='purple')
    ax.fill_between(variance_by_year.index, variance_by_year[feature], alpha=0.3, color='purple')
    ax.set_xlabel('Year')
    ax.set_ylabel('Variance')
    ax.set_title(f'{feature.replace("_", " ").title()}', fontweight='bold', fontsize=10)
    ax.grid(True, alpha=0.3)

axes[-1].axis('off')

plt.suptitle('Feature Variance Evolution (1972-2021) - Inter-District Variation', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('variance_evolution_50years.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Variance evolution plot saved")

## 4. National Trend Analysis

In [ ]:
features = ['Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Pig_per_HH', 'Poultry_per_HH']
national_trends = df_long.groupby('Year')[features].mean().reset_index()

fig, axes = plt.subplots(3, 3, figsize=(16, 12))
axes = axes.flatten()

for idx, feature in enumerate(features):
    ax = axes[idx]
    ax.plot(national_trends['Year'], national_trends[feature], linewidth=2, marker='o', markersize=2)
    ax.set_xlabel('Year')
    ax.set_ylabel(feature.replace('_', ' '))
    ax.set_title(feature.replace('_', ' ').title(), fontweight='bold')
    ax.grid(True, alpha=0.3)
    z = np.polyfit(national_trends['Year'], national_trends[feature], 1)
    p = np.poly1d(z)
    ax.plot(national_trends['Year'], p(national_trends['Year']), '--r', alpha=0.7)

for idx in range(len(features), len(axes)):
    axes[idx].axis('off')

plt.suptitle('Nepal Agricultural Trends: 1972-2021', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('national_trends_50years.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 50-year change analysis
year_1972 = national_trends[national_trends['Year'] == 1972].iloc[0]
year_2021 = national_trends[national_trends['Year'] == 2021].iloc[0]

print('50-YEAR CHANGE ANALYSIS (1972 -> 2021)')
print('='*60)
for feature in features:
    val_1972 = year_1972[feature]
    val_2021 = year_2021[feature]
    pct_change = ((val_2021 - val_1972) / (val_1972 + 0.01)) * 100
    direction = '+' if pct_change > 0 else ''
    print(f'{feature}: 1972={val_1972:.2f} -> 2021={val_2021:.2f} ({direction}{pct_change:.1f}%)')

## 5. Era-Based Clustering

In [ ]:
def assign_era(year):
    if year <= 1985:
        return 'Era 1: Traditional (1972-1985)'
    elif year <= 2000:
        return 'Era 2: Early Modern (1986-2000)'
    else:
        return 'Era 3: Commercial (2001-2021)'

df_long['Era'] = df_long['Year'].apply(assign_era)

era_profiles = df_long.groupby(['Districts', 'Era'])[features].mean().reset_index()
print('Era Profiles (National Averages):')
print(era_profiles.groupby('Era')[features].mean().round(2))

### 5.1 Determine Optimal Number of Clusters (Elbow Method)


In [ ]:
# Elbow Method and Silhouette Score Analysis - Using Full 50-Year Dataset
print("=" * 70)
print("CLUSTERING ANALYSIS ON FULL 50-YEAR DATASET")
print("=" * 70)

clustering_features = ['Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Poultry_per_HH']

# Use full dataset for clustering
X_full = df_long[clustering_features].fillna(0)

scaler = StandardScaler()
X_scaled_full = scaler.fit_transform(X_full)

print(f"Clustering on {len(X_full):,} data points (50 years x {df_long['Districts'].nunique()} districts)")

# Calculate metrics for different k values
k_range = range(2, 11)
inertias = []
silhouette_scores = []

print("\nEvaluating K-Means for k = 2 to 10 on full dataset...")
print("=" * 60)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled_full)
    inertias.append(kmeans.inertia_)
    sil_score = silhouette_score(X_scaled_full, kmeans.labels_)
    silhouette_scores.append(sil_score)
    print(f"k={k}: Inertia={kmeans.inertia_:.2f}, Silhouette Score={sil_score:.3f}")

In [ ]:
# Plot Elbow Curve and Silhouette Scores for Full 50-Year Dataset
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elbow Curve
axes[0].plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('Inertia (Within-cluster sum of squares)', fontsize=12)
axes[0].set_title('Elbow Method for Optimal k (50-Year Dataset)', fontsize=14, fontweight='bold')
axes[0].set_xticks(list(k_range))
axes[0].grid(True, alpha=0.3)
axes[0].axvline(x=4, color='r', linestyle='--', alpha=0.7, label='Suggested k=4')
axes[0].legend()

# Silhouette Score
axes[1].plot(k_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score for Different k (50-Year Dataset)', fontsize=14, fontweight='bold')
axes[1].set_xticks(list(k_range))
axes[1].grid(True, alpha=0.3)

best_k_sil = list(k_range)[np.argmax(silhouette_scores)]
axes[1].axvline(x=best_k_sil, color='r', linestyle='--', alpha=0.7, label=f'Best k={best_k_sil}')
axes[1].legend()

plt.tight_layout()
plt.savefig('elbow_silhouette_50years.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nBest k based on Silhouette Score: {best_k_sil}")

### 5.2 Fit K-Means with Optimal k and Characterize Clusters


In [ ]:
# Cluster Full 50-Year Dataset
try:
    OPTIMAL_K = best_k_sil
    print(f"Using optimal k={OPTIMAL_K} from silhouette analysis")
except NameError:
    OPTIMAL_K = 4
    print(f"best_k_sil not defined, defaulting to k={OPTIMAL_K}")

clustering_features = ['Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Buffalo_per_HH', 'Goat_per_HH', 'Poultry_per_HH']

print("=" * 70)
print("K-MEANS CLUSTERING ON FULL 50-YEAR DATASET")
print("=" * 70)

# Fit KMeans on full dataset
kmeans_full = KMeans(n_clusters=OPTIMAL_K, random_state=42, n_init=10)
df_long['Cluster'] = kmeans_full.fit_predict(X_scaled_full)

# Dynamic Cluster Naming based on Feature Characteristics
print("Generating dynamic cluster names based on feature profiles...")
cluster_centers = kmeans_full.cluster_centers_
profile_names = {}

for i in range(OPTIMAL_K):
    center = cluster_centers[i]
    # Sort features by importance (Z-score) for this cluster
    sorted_idx = np.argsort(center)[::-1]
    
    # Identify dominant features (Z-score > 0.4 as threshold for distinctive high value)
    dominant_features = []
    for idx in sorted_idx:
        if center[idx] > 0.4:
            feat = clustering_features[idx]
            if feat == 'Poultry_per_HH': dominant_features.append('Commercial Poultry')
            elif feat == 'Goat_per_HH': dominant_features.append('Highland/Goat')
            elif feat == 'Buffalo_per_HH': dominant_features.append('Buffalo/Dairy')
            elif feat == 'Cattle_per_HH': dominant_features.append('Cattle')
            elif feat == 'Avg_Land_Size': dominant_features.append('Large Land')
            elif feat == 'Pct_Irrigated': dominant_features.append('Irrigated')
            elif feat == 'Pig_per_HH': dominant_features.append('Pig Farming')
    
    if not dominant_features:
        # If no strong positive features, check if it's average or low input
        if np.all(center < -0.4):
            name = "Subsistence / Low Input"
        else:
            name = "Mixed / Diversified"
    else:
        name = " & ".join(dominant_features[:2]) # Combine top 2
        if len(dominant_features) > 2:
            name += "+"
            
    profile_names[i] = name

print("Assigned Cluster Names:")
for k, v in profile_names.items():
    print(f"  Cluster {k}: {v}")

df_long['Profile'] = df_long['Cluster'].map(profile_names)

print(f"\nClustering Results (Full 50-Year Dataset):")
print(f"Total data points: {len(df_long):,}")
print(f"Silhouette Score: {silhouette_score(X_scaled_full, df_long['Cluster']):.3f}")
print(f"\nCluster Distribution:")
print(df_long['Cluster'].value_counts().sort_index())

# Also update 2021 subset for reference
df_2021_full = df_long[df_long['Year'] == 2021].copy()
X_2021 = df_2021_full[clustering_features].fillna(0)
X_2021_scaled = scaler.transform(X_2021)
print(f"\n2021 Cluster Distribution:")
print(df_2021_full['Cluster'].value_counts().sort_index())

In [ ]:
# Characterize clusters with detailed profiles - Full 50-Year Dataset
print("=" * 80)
print("CLUSTER PROFILES (FULL 50-YEAR DATASET)")
print("=" * 80)

cluster_profiles_full = df_long.groupby('Cluster')[clustering_features].mean()

print("\nCluster Profiles (Mean Values Across All 50 Years):")
display(cluster_profiles_full.round(2))

# Cluster distribution by Era
print("\n" + "=" * 80)
print("CLUSTER DISTRIBUTION BY ERA")
print("=" * 80)
cluster_era = pd.crosstab(df_long['Era'], df_long['Cluster'], normalize='index') * 100
display(cluster_era.round(1))

# Detailed cluster analysis
print("\n" + "=" * 80)
print("DETAILED CLUSTER ANALYSIS (50-YEAR DATASET)")
print("=" * 80)

for cluster_id in range(OPTIMAL_K):
    cluster_data = df_long[df_long['Cluster'] == cluster_id]
    cluster_2021 = df_2021_full[df_2021_full['Cluster'] == cluster_id]
    
    print(f"\n{'─' * 60}")
    print(f"CLUSTER {cluster_id}: {profile_names[cluster_id]}")
    print(f"{'─' * 60}")
    print(f"Total Data Points (50 years): {len(cluster_data):,}")
    print(f"Districts in 2021: {len(cluster_2021)}")
    
    # Show districts in 2021
    if len(cluster_2021) > 0:
        district_names = cluster_2021['Districts'].str.strip().tolist()[:8]
        print(f"Sample Districts (2021): {', '.join(district_names)}")
        if len(cluster_2021) > 8:
            print(f"  ... and {len(cluster_2021) - 8} more")
    
    print(f"\nKey Characteristics (Mean across 50 years):")
    for feat in clustering_features:
        print(f"  - {feat}: {cluster_data[feat].mean():.3f}")

In [ ]:
# Cluster Visualization: Box plots for features across clusters (Full 50-Year Dataset)
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, feature in enumerate(clustering_features):
    sns.boxplot(x='Cluster', y=feature, data=df_long, ax=axes[idx], palette='husl')
    axes[idx].set_title(f'{feature.replace("_", " ").title()} (50-Year Data)', fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Cluster')
    axes[idx].set_ylabel('')

plt.suptitle('Feature Distribution Across Clusters (Full 50-Year Dataset)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('cluster_boxplots_50years.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cluster Heatmap - Normalized profiles (Full 50-Year Dataset)
cluster_means_full = df_long.groupby('Cluster')[clustering_features].mean()
cluster_means_normalized = (cluster_means_full - cluster_means_full.min()) / (cluster_means_full.max() - cluster_means_full.min())

plt.figure(figsize=(14, 8))
sns.heatmap(cluster_means_normalized.T, annot=cluster_means_full.T.round(2), 
            cmap='YlOrRd', fmt='.2f', linewidths=0.5,
            xticklabels=[f'C{i}: {profile_names[i][:15]}' for i in range(OPTIMAL_K)],
            yticklabels=[f.replace('_', ' ').title() for f in clustering_features])
plt.title('Cluster Profiles: Feature Comparison Heatmap (50-Year Dataset)', fontsize=14, fontweight='bold')
plt.xlabel('Cluster (Profile)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.savefig('cluster_heatmap_50years.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cluster Evolution Over Time
print("=" * 70)
print("CLUSTER EVOLUTION ANALYSIS (1972-2021)")
print("=" * 70)

# Calculate cluster distribution by year
cluster_by_year = df_long.groupby(['Year', 'Cluster']).size().unstack(fill_value=0)
cluster_by_year_pct = cluster_by_year.div(cluster_by_year.sum(axis=1), axis=0) * 100

# Plot cluster evolution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Absolute counts
cluster_by_year.plot(kind='area', stacked=True, ax=axes[0], 
                     colormap='Set2', alpha=0.8)
axes[0].set_xlabel('Year', fontsize=12)
axes[0].set_ylabel('Number of Districts', fontsize=12)
axes[0].set_title('Cluster Distribution Over Time (Count)', fontsize=14, fontweight='bold')
axes[0].legend(title='Cluster', labels=[f'C{i}: {profile_names[i][:20]}' for i in range(OPTIMAL_K)], 
               loc='upper left', fontsize=8)

# Percentage
cluster_by_year_pct.plot(kind='area', stacked=True, ax=axes[1], 
                         colormap='Set2', alpha=0.8)
axes[1].set_xlabel('Year', fontsize=12)
axes[1].set_ylabel('Percentage of Districts', fontsize=12)
axes[1].set_title('Cluster Distribution Over Time (Percentage)', fontsize=14, fontweight='bold')
axes[1].legend(title='Cluster', labels=[f'C{i}: {profile_names[i][:20]}' for i in range(OPTIMAL_K)], 
               loc='upper left', fontsize=8)

plt.tight_layout()
plt.savefig('cluster_evolution_50years.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Cluster evolution plot saved")

In [ ]:
# Cluster Transition Analysis: Evolution of District Farming Profiles (1972-2021)
print("=" * 70)
print("CLUSTER TRANSITION ANALYSIS: EVOLUTION OF FARMING PROFILES (1972-2021)")
print("=" * 70)

# Compare 1972 vs 2021 clusters for each district
df_1972 = df_long[df_long['Year'] == 1972][['Districts', 'Cluster']].rename(columns={'Cluster': 'Cluster_1972'})
df_2021_clusters = df_long[df_long['Year'] == 2021][['Districts', 'Cluster']].rename(columns={'Cluster': 'Cluster_2021'})

transition_df = df_1972.merge(df_2021_clusters, on='Districts')
transition_matrix = pd.crosstab(transition_df['Cluster_1972'], transition_df['Cluster_2021'])

print("\nCluster Transition Matrix (1972 -> 2021):")
display(transition_matrix)

# Visualize transition matrix
plt.figure(figsize=(10, 8))
sns.heatmap(transition_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=[f'2021 C{i}' for i in range(OPTIMAL_K)],
            yticklabels=[f'1972 C{i}' for i in range(OPTIMAL_K)])
plt.title('Cluster Transition Matrix: 1972 → 2021', fontsize=14, fontweight='bold')
plt.xlabel('Cluster in 2021', fontsize=12)
plt.ylabel('Cluster in 1972', fontsize=12)
plt.tight_layout()
plt.savefig('cluster_transition_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Calculate stability
stable_districts = (transition_df['Cluster_1972'] == transition_df['Cluster_2021']).sum()
print(f"\nCluster Stability Analysis:")
print(f"Districts that remained in same cluster: {stable_districts} ({stable_districts/len(transition_df)*100:.1f}%)")
print(f"Districts that changed clusters: {len(transition_df) - stable_districts} ({(len(transition_df) - stable_districts)/len(transition_df)*100:.1f}%)")

## 6. Structural Change Detection

In [ ]:
def calculate_structural_change(district_data):
    first_decade = district_data[district_data['Year'] <= 1982]
    last_decade = district_data[district_data['Year'] >= 2012]
    if len(first_decade) == 0 or len(last_decade) == 0:
        return 0
    changes = []
    for col in clustering_features:
        early_mean = first_decade[col].mean()
        late_mean = last_decade[col].mean()
        if early_mean > 0:
            changes.append(abs((late_mean - early_mean) / early_mean))
    return np.mean(changes) * 100 if changes else 0

structural_changes = []
for district in df_long['Districts'].unique():
    district_data = df_long[df_long['Districts'] == district]
    change_index = calculate_structural_change(district_data)
    structural_changes.append({'District': district, 'Structural_Change_Index': change_index})

df_structural = pd.DataFrame(structural_changes).sort_values('Structural_Change_Index', ascending=False)
print('Top 15 Districts by Structural Change:')
print(df_structural.head(15).to_string(index=False))

In [ ]:
# Visualize structural change
fig, ax = plt.subplots(figsize=(12, 8))
top_20 = df_structural.head(20)
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(top_20)))
ax.barh(top_20['District'], top_20['Structural_Change_Index'], color=colors)
ax.set_xlabel('Structural Change Index (%)')
ax.set_title('Top 20 Districts by Structural Change (1972-2021)', fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.savefig('structural_change_ranking.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Forecasting (2022-2030)

In [ ]:
forecast_years = list(range(2022, 2031))

forecast_df = pd.DataFrame({'Year': forecast_years})
for feature in features:
    model = LinearRegression()
    model.fit(national_trends['Year'].values.reshape(-1, 1), national_trends[feature].values)
    preds = model.predict(np.array(forecast_years).reshape(-1, 1))
    forecast_df[feature] = np.clip(preds, 0, None)

print('National Forecast (2022-2030):')
print(forecast_df.round(2))
forecast_df.to_csv('nepal_agriculture_forecast_2030.csv', index=False)

In [ ]:
# Visualize forecast
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()
plot_features = ['Poultry_per_HH', 'Avg_Land_Size', 'Pct_Irrigated', 'Cattle_per_HH', 'Goat_per_HH', 'Pig_per_HH']

for idx, feature in enumerate(plot_features):
    ax = axes[idx]
    ax.plot(national_trends['Year'], national_trends[feature], 'b-', linewidth=2, label='Historical')
    ax.plot(forecast_df['Year'], forecast_df[feature], 'r--', linewidth=2, label='Forecast')
    ax.fill_between(forecast_df['Year'], forecast_df[feature]*0.9, forecast_df[feature]*1.1, alpha=0.2, color='red')
    ax.set_xlabel('Year')
    ax.set_title(feature.replace('_', ' ').title(), fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axvline(x=2021, color='gray', linestyle=':', alpha=0.7)

plt.suptitle('Agricultural Trends: Historical + Forecast', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('trends_with_forecast.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Decision Tree Classification

In [ ]:
# Prepare data for classification - Using Full 50-Year Dataset
print("=" * 70)
print("DECISION TREE CLASSIFICATION ON FULL 50-YEAR DATASET")
print("=" * 70)

X_clf = df_long[clustering_features].copy()
y_clf = df_long['Cluster'].copy()

print(f"\nClassification Dataset (Full 50 Years):")
print(f"  Features shape: {X_clf.shape}")
print(f"  Total samples: {len(X_clf):,}")
print(f"\nTarget distribution:")
print(y_clf.value_counts().sort_index())

# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X_clf, y_clf, 
    test_size=0.2, 
    random_state=42,
    stratify=y_clf  # Maintain cluster proportions
)

print(f"\nTraining set: {X_train.shape[0]:,} samples")
print(f"Testing set: {X_test.shape[0]:,} samples")
print(f"\nTraining set distribution:")
print(y_train.value_counts().sort_index())
print(f"\nTesting set distribution:")
print(y_test.value_counts().sort_index())

# Train Decision Tree Classifier
dt = DecisionTreeClassifier(
    max_depth=5,              # Slightly deeper for more data
    min_samples_split=10,     # Adjusted for larger dataset
    min_samples_leaf=5,       # Adjusted for larger dataset
    random_state=42
)

# Fit the model
dt.fit(X_train, y_train)

print("\n✓ Decision Tree Classifier trained on full 50-year dataset!")
print(f"\nTree depth: {dt.get_depth()}")
print(f"Number of leaves: {dt.get_n_leaves()}")

In [ ]:
# Model Evaluation - Full 50-Year Dataset
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("=" * 60)
print("MODEL PERFORMANCE (50-YEAR DATASET)")
print("=" * 60)
print(f"Training Accuracy: {train_accuracy:.2%}")
print(f"Testing Accuracy:  {test_accuracy:.2%}")
print(f"Training Samples:  {len(y_train):,}")
print(f"Testing Samples:   {len(y_test):,}")

# Detailed Classification Report
print("\n" + "=" * 60)
print("CLASSIFICATION REPORT (TEST SET)")
print("=" * 60)
target_names = [f"C{i}: {profile_names[i][:20]}" for i in range(OPTIMAL_K)]
print(classification_report(y_test, y_test_pred, target_names=target_names))

In [ ]:
# Confusion Matrix - Full 50-Year Dataset
cm = confusion_matrix(y_test, y_test_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[f'C{i}' for i in range(OPTIMAL_K)],
            yticklabels=[f'C{i}' for i in range(OPTIMAL_K)])
plt.xlabel('Predicted Cluster', fontsize=12)
plt.ylabel('Actual Cluster', fontsize=12)
plt.title('Confusion Matrix - Decision Tree (50-Year Dataset)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('confusion_matrix_50years.png', dpi=150, bbox_inches='tight')
plt.show()

# Calculate per-class metrics
print("\nPer-Class Accuracy:")
for i in range(OPTIMAL_K):
    class_mask = y_test == i
    class_acc = accuracy_score(y_test[class_mask], y_test_pred[class_mask])
    print(f"  Cluster {i} ({profile_names[i][:20]}): {class_acc:.2%}")

In [ ]:
# Visualize the Decision Tree - Full 50-Year Dataset
plt.figure(figsize=(24, 14))
plot_tree(
    dt, 
    feature_names=clustering_features,
    class_names=[profile_names[i] for i in range(OPTIMAL_K)],
    filled=True,
    rounded=True,
    fontsize=9,
    proportion=True
)
plt.title('Decision Tree for Livestock Farming Profile Classification (50-Year Dataset)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('decision_tree_50years.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Extract and display decision rules in text format
print("Decision Tree Rules:")
print("=" * 80)
tree_rules = export_text(
    dt, 
    feature_names=clustering_features
)
print(tree_rules)


In [ ]:
# Feature Importance from Decision Tree - Full 50-Year Dataset
feature_importance = pd.DataFrame({
    'feature': clustering_features,
    'importance': dt.feature_importances_
}).sort_values('importance', ascending=True)

# Plot feature importance
plt.figure(figsize=(10, 8))
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(feature_importance)))
plt.barh(feature_importance['feature'], feature_importance['importance'], color=colors)
plt.xlabel('Feature Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance in Decision Tree (50-Year Dataset)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('feature_importance_50years.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFeature Importance Ranking:")
print("=" * 50)
for _, row in feature_importance.sort_values('importance', ascending=False).iterrows():
    print(f"  {row['feature']}: {row['importance']:.4f} ({row['importance']*100:.1f}%)")

In [ ]:
# Model Performance Analysis by Era
print("=" * 70)
print("CLASSIFICATION PERFORMANCE BY ERA")
print("=" * 70)

# Add Era to test data for analysis
test_indices = X_test.index
df_test = df_long.loc[test_indices].copy()
df_test['Predicted'] = y_test_pred

# Calculate accuracy by Era
era_accuracy = []
for era in df_test['Era'].unique():
    era_mask = df_test['Era'] == era
    era_acc = accuracy_score(df_test[era_mask]['Cluster'], df_test[era_mask]['Predicted'])
    era_count = era_mask.sum()
    era_accuracy.append({'Era': era, 'Accuracy': era_acc, 'Samples': era_count})
    print(f"{era}: Accuracy = {era_acc:.2%} ({era_count} samples)")

# Visualize accuracy by era
era_df = pd.DataFrame(era_accuracy)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy by Era
axes[0].bar(era_df['Era'], era_df['Accuracy'], color=['coral', 'steelblue', 'forestgreen'])
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('Classification Accuracy by Era', fontsize=14, fontweight='bold')
axes[0].set_ylim(0, 1)
axes[0].tick_params(axis='x', rotation=15)
for i, (acc, era) in enumerate(zip(era_df['Accuracy'], era_df['Era'])):
    axes[0].text(i, acc + 0.02, f'{acc:.1%}', ha='center', fontsize=10)

# Sample distribution by Era
axes[1].bar(era_df['Era'], era_df['Samples'], color=['coral', 'steelblue', 'forestgreen'])
axes[1].set_ylabel('Number of Test Samples', fontsize=12)
axes[1].set_title('Test Sample Distribution by Era', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.savefig('classification_by_era_50years.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Policy Recommendations and Vulnerability Assessment


In [ ]:
# Generate policy recommendations based on cluster characteristics
print("\n" + "=" * 80)
print("POLICY RECOMMENDATIONS BY FARMING PROFILE")
print("=" * 80)

recommendations = {
    0: {
        'vulnerability': 'LOW-MEDIUM',
        'strengths': [
            'Good irrigation infrastructure',
            'Diversified livestock portfolio',
            'Access to markets'
        ],
        'challenges': [
            'Market price volatility',
            'Disease outbreak risks in dense populations'
        ],
        'recommendations': [
            'Establish livestock insurance schemes',
            'Develop cold storage and processing facilities',
            'Implement disease surveillance systems',
            'Promote cooperative marketing'
        ]
    },
    1: {
        'vulnerability': 'HIGH',
        'strengths': [
            'Traditional farming knowledge',
            'Low input dependency'
        ],
        'challenges': [
            'Limited irrigation',
            'Small land holdings',
            'Climate vulnerability',
            'Limited market access'
        ],
        'recommendations': [
            'Implement small-scale irrigation projects',
            'Provide goat/sheep farming subsidies',
            'Establish community pasture management',
            'Create mobile veterinary services',
            'Develop micro-credit programs'
        ]
    },
    2: {
        'vulnerability': 'MEDIUM-HIGH',
        'strengths': [
            'Large cattle holdings',
            'Highland adapted breeds',
            'Pastoral traditions'
        ],
        'challenges': [
            'Harsh climate conditions',
            'Limited infrastructure',
            'Seasonal migration patterns'
        ],
        'recommendations': [
            'Support traditional transhumance practices',
            'Improve mountain road connectivity',
            'Establish highland breed conservation programs',
            'Create seasonal veterinary camps',
            'Develop high-altitude fodder cultivation'
        ]
    },
    3: {
        'vulnerability': 'MEDIUM',
        'strengths': [
            'Diversified farming systems',
            'Moderate irrigation access',
            'Mixed livestock-crop integration'
        ],
        'challenges': [
            'Land fragmentation',
            'Labor migration',
            'Limited mechanization'
        ],
        'recommendations': [
            'Promote integrated farming systems',
            'Support small-scale dairy cooperatives',
            'Provide training in improved animal husbandry',
            'Develop local feed production',
            'Create farmer producer organizations'
        ]
    }
}

# Default recommendation for additional clusters
default_rec = {
    'vulnerability': 'UNCERTAIN',
    'strengths': ['Emerging farming profile', 'Specific adaptation to local conditions'],
    'challenges': ['Need for detailed assessment', 'Potential resource constraints'],
    'recommendations': [
        'Conduct detailed local needs assessment',
        'Develop tailored intervention strategies',
        'Monitor production trends closely'
    ]
}

for cluster_id in range(OPTIMAL_K):
    rec = recommendations.get(cluster_id, default_rec)
    
    # Get profile name safely
    profile_name = profile_names.get(cluster_id, f"Cluster {cluster_id}")
    
    print(f"\n{'─' * 60}")
    print(f"CLUSTER {cluster_id}: {profile_name}")
    print(f"{'─' * 60}")
    print(f"\nVulnerability Level: {rec['vulnerability']}")
    
    print(f"\nStrengths:")
    for s in rec['strengths']:
        print(f"  ✓ {s}")
    
    print(f"\nChallenges:")
    for c in rec['challenges']:
        print(f"  ✗ {c}")
    
    print(f"\nPolicy Recommendations:")
    for i, r in enumerate(rec['recommendations'], 1):
        print(f"  {i}. {r}")

## 10. Conclusion and Summary

In [ ]:
# Comprehensive Summary - Updated for Full 50-Year Dataset Analysis
print("""
================================================================================
                              CONCLUSION
================================================================================

This study successfully applied machine learning techniques to analyze 50 years
of livestock farming data across Nepal's districts (1972-2021), using both
the original 2021 dataset and synthetic historical data.

KEY ACHIEVEMENTS:

1. DATA INTEGRATION & COMPREHENSIVE EDA:
   - Original 2021 baseline data from 3 sources (Holdings, Irrigation, Livestock)
   - Generated synthetic historical data for 50 years (1972-2020)
   - Comprehensive EDA on both original and synthetic datasets
   - Feature engineering with 7 key derived indicators

2. TEMPORAL ANALYSIS (FULL 50-YEAR DATASET):
   - Correlation analysis across all 50 years
   - Distribution evolution visualization (1970s vs 1990s vs 2010s)
   - Decade-wise statistical summaries and growth rates
   - Feature variance evolution over time
   - District-level time series analysis

3. CLUSTERING ANALYSIS (FULL 50-YEAR DATASET):
""")

print(f"   - Analyzed {len(df_long):,} data points (50 years x {df_long['Districts'].nunique()} districts)")
print(f"   - Identified {OPTIMAL_K} distinct farming profiles using K-Means")
sil_full = silhouette_score(X_scaled_full, df_long['Cluster'])
print(f"   - Achieved silhouette score of {sil_full:.3f}")
print("   - Profiles: Commercial Hubs, Subsistence Mixed, Highland Pastoral, Smallholder Diversified")
print("   - Cluster evolution analysis over 50 years")
print("   - Cluster transition matrix (1972 → 2021)")

print(f"""
4. CLASSIFICATION MODEL (FULL 50-YEAR DATASET):
   - Training samples: {len(y_train):,}
   - Testing samples: {len(y_test):,}
   - Training Accuracy: {train_accuracy:.1%}
   - Testing Accuracy: {test_accuracy:.1%}
   - Model provides interpretable rules for profile classification
   - Era-wise performance analysis
   - Key factors identified through feature importance analysis

5. FORECASTING:
   - Linear trend projection to 2030
   - Identified continued poultry growth and land pressure trends

6. POLICY IMPLICATIONS:
   - Vulnerability assessment for each farming profile
   - Targeted recommendations for agricultural interventions
   - Evidence-based framework for resource allocation

================================================================================
""")

print("=" * 70)
print("FILES GENERATED:")
print("=" * 70)
output_files = [
    # Data files
    ('nepal_agriculture_50_years.csv', '50 years of synthetic district-level data'),
    ('nepal_agriculture_forecast_2030.csv', 'National forecast to 2030'),
    
    # Original data visualizations
    ('original_data_distributions.png', 'Original 2021 dataset distributions'),
    ('temporal_feature_correlation_2021.png', '2021 baseline correlation heatmap'),
    ('temporal_feature_distributions_2021.png', '2021 feature distribution plots'),
    ('temporal_pairplot_2021.png', '2021 pairwise relationships'),
    
    # 50-year EDA visualizations
    ('correlation_matrix_50years.png', 'Full dataset correlation heatmap'),
    ('distribution_evolution_50years.png', 'Distribution evolution by decade'),
    ('boxplot_by_decade_50years.png', 'Feature boxplots by decade'),
    ('pairplot_by_era_50years.png', 'Pairplot colored by era'),
    ('district_trends_50years.png', 'District time series'),
    ('variance_evolution_50years.png', 'Feature variance over time'),
    
    # Clustering visualizations
    ('elbow_silhouette_50years.png', 'Cluster selection analysis'),
    ('cluster_boxplots_50years.png', 'Feature distributions by cluster'),
    ('cluster_heatmap_50years.png', 'Cluster profile heatmap'),
    ('cluster_evolution_50years.png', 'Cluster distribution over time'),
    ('cluster_transition_matrix.png', 'Cluster transitions 1972-2021'),
    
    # Trend visualizations
    ('national_trends_50years.png', '50-year trend analysis'),
    ('structural_change_ranking.png', 'District structural change ranking'),
    ('trends_with_forecast.png', 'Historical trends with forecast'),
    
    # Classification visualizations
    ('confusion_matrix_50years.png', 'Classification confusion matrix'),
    ('decision_tree_50years.png', 'Decision tree visualization'),
    ('feature_importance_50years.png', 'Feature importance ranking'),
    ('classification_by_era_50years.png', 'Classification accuracy by era')
]

for filename, description in output_files:
    print(f"  • {filename}: {description}")

print(f"\n" + "=" * 70)
print("SUMMARY STATISTICS")
print("=" * 70)
print(f"Total data points analyzed: {len(df_long):,}")
print(f"Original 2021 data points: {len(df_2021)}")
print(f"Years covered: 1972-2021 (50 years)")
print(f"Districts: {df_long['Districts'].nunique()}")
print(f"Features: {len(features)}")
print(f"Clustering Features: {len(clustering_features)}")
print(f"Optimal Clusters: {OPTIMAL_K}")
print(f"Classification Accuracy: {test_accuracy:.1%}")

print("\n✓ COMPLETE 50-YEAR TEMPORAL ANALYSIS FINISHED!")